# Stocker sous HDF5

De façon à simplifier l'importation des images, j'ai fait le choix d'utiliser un fichier hdf5 plutôt que d'importer les images depuis le dossier. Les répertoires normaux prennent plus de place, sont inefficaces pour l'itération, ne supportent pas les entrées/sorties parallèles, l'accès aléatoire, sont non hétérogènes. La démarche adoptée ici est de lire chaque image dans le dossier, puis de l'envoyer vers une fonction de prétraitement puis de l'analyser pour obtenir un tableau numérique global qui pourra ensuite être utilisé pour créer un fichier hdf5.

In [1]:
from random import shuffle
import glob
import pandas as pd
import numpy as np
import h5py
import cv2

#sélectionner le dossier à transformer 'train' ou 'test'
folder = 'test'

#déterminer une valeur pour recadrer l'image (max 2070px)
IMAGE_RESIZED = 256

In [68]:
#FILE PATH
save_path = folder+'_dataset.hdf5'
img_path = folder+'_dataset/*.jpg'

files = glob.glob(img_path)

In [69]:
print("Nombre de fichiers : {}".format(len(files)))

Nombre de fichiers : 1989


In [70]:
# IMPORTER LES LABELS
labels = pd.read_csv(folder+'_data_labels.csv')
labels['images'] = folder+'_dataset/' + labels['images'].astype(str)
labels.head()

,images,labels
0,test_dataset/AE00022_095817_00_1_1_2001.jpg,NaN
1,test_dataset/AE00382_081204_00_3_4_2001.jpg,NaN
2,test_dataset/AE00281_235123_00_2_1_2001.jpg,NaN
3,test_dataset/AE00408_051313_00_4_4_2001.jpg,NaN
4,test_dataset/AE00379_034020_00_1_2_2001.jpg,NaN


In [13]:
shape = (len(files), IMAGE_RESIZED, IMAGE_RESIZED, 3)

with h5py.File(save_path,'w') as f:
    f.create_dataset(folder+'_img', shape, np.uint8)
    f.create_dataset("labels", (len(files),), np.uint8)
    f["labels"][...] = labels['labels'] #stocker les labels
    
    for i in range(len(labels['images'])):
        
        if i % 1000 == 0 and i > 1:
            print (folder+' data: {}/{}'.format(i, len(files)) ) #pour suivre l'évolution

        path = labels.loc[i, 'images']
        img = cv2.imread(path)
        img = cv2.resize(img, (IMAGE_RESIZED, IMAGE_RESIZED), interpolation=cv2.INTER_CUBIC) # recadrer à la taille voulue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 charger les images en BGR puis convertir en RGB
        f[folder+'_img'][i, ...] = img[None] #stocker les images


train data: 1000/10609
train data: 2000/10609
train data: 3000/10609
train data: 4000/10609
train data: 5000/10609
train data: 6000/10609
train data: 7000/10609
train data: 8000/10609
train data: 9000/10609
train data: 10000/10609


# Stocker sous TFRecord

Faisant face à une saturation de la mémoire dès lors la taille des images dépassait 150x150px, j'ai décidé de formater mes images vers le format TFRecord, optimisé pour l'usage de Tensorflow et cela s'est avéré efficace. Pour en savoir plus sur ce format, je conseille de consulter cet [article](https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564). La source du code provient de ce [notebook](https://www.kaggle.com/cdeotte/how-to-create-tfrecords).

In [20]:
#CHARGEMENT DES LIBRAIRES
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math

#SELECTIONNER LE DOSSIER A FORMATER 'train' OU 'test'
folder = 'test'

#TAILLE DE L'IMAGE A RE-AJUSTER
IMAGE_RESIZED = 512

In [21]:
#CHEMIN VERS LES IMAGES
PATH = folder+'_dataset/'
IMGS = os.listdir(PATH)
print('There are %i train images'%(len(IMGS)))

There are 1989 train images


In [22]:
#IMPORTER LES LABELS
labels = pd.read_csv(folder+'_data_labels.csv')
labels['images'] = folder+'_dataset/' + labels['images'].astype(str)
labels['images'] = labels['images'].str.split(pat='/', expand=True)[1]
labels['images'] = labels['images'].str.split(pat='.', expand=True)[0]
labels['labels'] = labels['labels'].fillna(0)
labels['labels'] = labels['labels'].astype('int')
labels.head()

,images,labels
0,AE00022_095817_00_1_1_2001,0
1,AE00382_081204_00_3_4_2001,0
2,AE00281_235123_00_2_1_2001,0
3,AE00408_051313_00_4_4_2001,0
4,AE00379_034020_00_1_2_2001,0


In [23]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy()
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [24]:
def serialize_example(feature0, feature1, feature2):
  feature = {
      'image': _bytes_feature(feature0),
      'image_name': _bytes_feature(feature1),
      'label': _int64_feature(feature2)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [25]:
SIZE = 2071
CT = len(IMGS)//SIZE + int(len(IMGS)%SIZE!=0)
for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j,CT))
    CT2 = min(SIZE,len(IMGS)-j*SIZE)
    with tf.io.TFRecordWriter(folder+'%.2i-%i.tfrec'%(j,CT2)) as writer:
        for k in range(CT2):
            img = cv2.imread(PATH+IMGS[SIZE*j+k])
            img = cv2.resize(img, (IMAGE_RESIZED, IMAGE_RESIZED), interpolation=cv2.INTER_CUBIC)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # Fix incorrect colors
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tostring()
            name = IMGS[SIZE*j+k].split('.')[0]
            row = labels.loc[labels.images==name]
            example = serialize_example(img, str.encode(name), row.labels.values[0])
            writer.write(example)
            if k%100==0: print(k,', ',end='')


Writing TFRecord 0 of 1...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 